In [1]:
%pip install --quiet mesa

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.6/65.6 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 29.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 39.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.9/105.9 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 34.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 51.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 55.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [9]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import joblib

iris = load_iris()
X = iris.data
y = iris.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

joblib.dump(clf, 'iris_classifier.joblib')

loaded_model = joblib.load('iris_classifier.joblib')

# For example, predict the first two samples from the test set
predictions = loaded_model.predict(X_test[:2])
print(predictions)

[1 0]


In [10]:
# Load Iris Dataset
iris = load_iris()
class_names = iris.target_names

# Split Data into Train and Test Sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize and Train the Classifier
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

# Save the Model to Disk
joblib.dump(clf, 'iris_classifier.joblib')

# Load the Model from Disk
loaded_model = joblib.load('iris_classifier.joblib')

# Make Predictions on Sample Data
# For example, predict the first two samples from the test set
predictions = loaded_model.predict(X_test[:2])

# Print Predictions with String Names
for prediction in predictions:
    print(class_names[prediction])

versicolor
setosa


In [11]:
X_test[:2]

array([[6.1, 2.8, 4.7, 1.2],
       [5.7, 3.8, 1.7, 0.3]])

In [15]:
from mesa import Agent, Model
from mesa.time import RandomActivation
import random
import joblib
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

class Informer(Agent):
    def __init__(self, unique_id, model, classifier_agent):
        super().__init__(unique_id, model)
        self.iris = load_iris()
        self.X_test = self.split_data()
        self.classifier_agent = classifier_agent

    def split_data(self):
        X = self.iris.data
        y = self.iris.target
        _, X_test, _, _ = train_test_split(X, y, test_size=0.2, random_state=42)
        return X_test

    def step(self):
        random_sample = random.choice(self.X_test)
        self.classifier_agent.predict_sample(random_sample)

class Classifier(Agent):
    def __init__(self, unique_id, model, evaluator_agent):
        super().__init__(unique_id, model)
        self.loaded_model = joblib.load('iris_classifier.joblib')
        self.evaluator_agent = evaluator_agent

    def predict_sample(self, sample):
        prediction = self.loaded_model.predict([sample])[0]
        self.evaluator_agent.evaluate(prediction)

class Evaluator(Agent):
    def __init__(self, unique_id, model):
        super().__init__(unique_id, model)

    def evaluate(self, prediction):
        if prediction == 0:
            print("Iris setosa")
        elif prediction == 1:
            print("Iris versicolor")
        else:
            print("Iris virginica")

class IrisModel(Model):
    def __init__(self, num_informers, num_classifiers, num_evaluators):
        self.num_informers = num_informers
        self.num_classifiers = num_classifiers
        self.num_evaluators = num_evaluators
        self.schedule = RandomActivation(self)

        # Create agents
        evaluator_agent = Evaluator(0, self)
        self.schedule.add(evaluator_agent)

        classifier_agent = Classifier(1, self, evaluator_agent)
        self.schedule.add(classifier_agent)

        for i in range(self.num_informers):
            informer = Informer(i, self, classifier_agent)
            self.schedule.add(informer)

    def step(self):
        self.schedule.step()

# Create and run the model
model = IrisModel(num_informers=1, num_classifiers=1, num_evaluators=1)
for i in range(10):  # Run for 10 steps
    model.step()

<ipython-input-15-17db1e759acd>:37: FutureWarning: The Mesa Model class was not initialized. In the future, you need to explicitly initialize the Model by calling super().__init__() on initialization.
  super().__init__(unique_id, model)


Iris virginica
Iris versicolor
Iris setosa
Iris versicolor
Iris versicolor
Iris versicolor
Iris virginica
Iris virginica
Iris versicolor
Iris setosa


## **DRAFT**

### **Informer anf Data Preprocessor**

In [2]:
from mesa import Agent, Model
from mesa.time import RandomActivation

In [ ]:
class Informer_Agent(Agent):
    def __init__(self, unique_id, model, message):
        super().__init__(unique_id, model)
        self.message = message

    def step(self):
        # Submit the message to the data preprocessor agent
        self.model.data_preprocessor.receive_message(self.message)

In [ ]:
class DataPreprocessorAgent(Agent):
    def __init__(self, unique_id, model):
        super().__init__(unique_id, model)

    def receive_message(self, message):
        # Here you can process the received message
        print("Received message:", message)

In [ ]:
import keras
import numpy as np
import pandas as pd
import tensorflow as tf
from keras import layers, Input, Model, ops
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

data = pd.read_csv('/content/twitter_dataset.csv')
df = data[['text', 'is_propaganda']]

import keras
from keras import layers

import keras
from keras import layers

@keras.utils.register_keras_serializable(package='Custom', name='TransformerBlock')
class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, trainable=True, rate=0.1, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        self.ff_dim = ff_dim
        self.trainable = trainable

        self.att = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = keras.Sequential([
            layers.Dense(ff_dim, activation="relu"),
            layers.Dense(embed_dim),
        ])
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    def call(self, inputs):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output)
        return self.layernorm2(out1 + ffn_output)

    def get_config(self):
        config = super().get_config()
        config.update({
            'embed_dim': self.embed_dim,
            'num_heads': self.num_heads,
            'ff_dim': self.ff_dim,
            'trainable': self.trainable,
        })
        return config


@keras.utils.register_keras_serializable(package='Custom', name='TokenAndPositionEmbedding')
class TokenAndPositionEmbedding(layers.Layer):
    def __init__(self, maxlen, vocab_size, embed_dim, trainable=True, **kwargs):
        super().__init__(**kwargs)
        self.maxlen = maxlen
        self.vocab_size = vocab_size
        self.embed_dim = embed_dim
        self.trainable = trainable

    def build(self, input_shape):
        self.token_emb = layers.Embedding(input_dim=self.vocab_size, output_dim=self.embed_dim, trainable=self.trainable)
        self.pos_emb = layers.Embedding(input_dim=self.maxlen, output_dim=self.embed_dim, trainable=self.trainable)
        super().build(input_shape)

    def call(self, x):
        maxlen = tf.shape(x)[-1]
        positions = tf.range(start=0, limit=self.maxlen, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x + positions

    def get_config(self):
        config = super().get_config()
        config.update({
            'maxlen': self.maxlen,
            'vocab_size': self.vocab_size,
            'embed_dim': self.embed_dim,
            'trainable': self.trainable
        })
        return config

    @classmethod
    def from_config(cls, config):
        return cls(**config)



vocab_size = 20000  # Only consider the top 20k words
maxlen = 200  # Only consider the first 200 words of each movie review

features = df['text']
labels = df['is_propaganda']

# Tokenize the text data
tokenizer = Tokenizer(num_words=vocab_size)
tokenizer.fit_on_texts(features)

# Convert text to sequences
sequences = tokenizer.texts_to_sequences(features)

# Pad sequences to ensure uniform length
padded_sequences = pad_sequences(sequences, maxlen=maxlen)

# Split the data into training and validation sets
x_train, x_val, y_train, y_val = train_test_split(padded_sequences, labels, test_size=0.2, random_state=42)
x_val, x_test, y_val, y_test = train_test_split(x_val, y_val, test_size=0.5, random_state=42)

embed_dim = 32  # Embedding size for each token
num_heads = 2  # Number of attention heads
ff_dim = 32  # Hidden layer size in feed forward network inside transformer

inputs = Input(shape=(maxlen,))
#embedding_layer = TokenAndPositionEmbedding(maxlen, vocab_size, embed_dim)
# Instantiate TokenAndPositionEmbedding with trainable argument
embedding_layer = TokenAndPositionEmbedding(maxlen, vocab_size, embed_dim, trainable=False)
x = embedding_layer(inputs)
transformer_block = TransformerBlock(embed_dim, num_heads, ff_dim)
x = transformer_block(x)
x = layers.GlobalAveragePooling1D()(x)
x = layers.Dropout(0.1)(x)
x = layers.Dense(60, activation="relu")(x)
x = layers.Dropout(0.1)(x)
x = layers.Dense(40, activation="relu")(x)
x = layers.Dropout(0.1)(x)
x = layers.Dense(20, activation="relu")(x)
x = layers.Dropout(0.1)(x)
outputs = layers.Dense(2, activation="softmax")(x)

model = Model(inputs=inputs, outputs=outputs)
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
history = model.fit(
    x_train, y_train, batch_size=32, epochs=20, validation_data=(x_val, y_val)
)

model.save("propaganda_classifier.keras")

In [ ]:
!pip install kaggle
from google.colab import files
files.upload()
!ls -lha kaggle.json
!pip install -q kaggle
!mkdir -p ~/.kaggle #Create the directory
!cp kaggle.json ~/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json
!kaggle datasets download -d bohdanmynzar/twitter-propaganda-classification
!unzip twitter-propaganda-classification.zip
!pip install --upgrade keras

In [4]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline

# Sample data
texts = ["I love this product!", "This is terrible.", "It's okay, not great."]

# Corresponding labels
labels = ["positive", "negative", "negative"]

# Preprocess data
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(texts)

# Train classifier
classifier = make_pipeline(CountVectorizer(), SVC(kernel='linear'))
classifier.fit(texts, labels)

# Test the classifier
test_texts = ["I'm happy with this purchase.", "Worst experience ever!"]
predictions = classifier.predict(test_texts)

# Output predictions
for text, prediction in zip(test_texts, predictions):
    print(f"Text: {text} \nPredicted Label: {prediction}\n")


Text: I'm happy with this purchase. 
Predicted Label: negative

Text: Worst experience ever! 
Predicted Label: negative



In [5]:
from mesa import Agent, Model
from mesa.time import BaseScheduler
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline

class InformerAgent(Agent):
    def __init__(self, unique_id, model, initial_string):
        super().__init__(unique_id, model)
        self.initial_string = initial_string

    def step(self):
        # Send the initial string to the preprocessor agent
        self.model.preprocessor_agent.receive_string(self.initial_string)

class PreprocessorAgent(Agent):
    def __init__(self, unique_id, model):
        super().__init__(unique_id, model)
        self.vectorizer = CountVectorizer()

    def preprocess_string(self, string):
        # Preprocess the string
        preprocessed_string = self.vectorizer.transform([string])
        return preprocessed_string

    def receive_string(self, string):
        preprocessed_string = self.preprocess_string(string)
        # Send the preprocessed string to the classifier agent
        self.model.classifier_agent.receive_preprocessed_string(preprocessed_string)

class ClassifierAgent(Agent):
    def __init__(self, unique_id, model):
        super().__init__(unique_id, model)
        self.classifier = make_pipeline(CountVectorizer(), SVC(kernel='linear'))

    def classify_string(self, preprocessed_string):
        # Classify the preprocessed string
        prediction = self.classifier.predict(preprocessed_string)[0]
        return prediction

    def receive_preprocessed_string(self, preprocessed_string):
        prediction = self.classify_string(preprocessed_string)
        # Send the prediction to the evaluator agent
        self.model.evaluator_agent.receive_prediction(prediction)

class EvaluatorAgent(Agent):
    def __init__(self, unique_id, model):
        super().__init__(unique_id, model)

    def evaluate_prediction(self, prediction):
        # Evaluate the prediction and display a message
        if prediction == "positive":
            print("The text is classified as positive.")
        elif prediction == "negative":
            print("The text is classified as negative.")
        else:
            print("The text cannot be classified.")

    def receive_prediction(self, prediction):
        self.evaluate_prediction(prediction)

class TextClassificationModel(Model):
    def __init__(self, initial_string):
        self.initial_string = initial_string
        self.scheduler = BaseScheduler(self)

        # Initialize agents
        self.informer_agent = InformerAgent(1, self, initial_string)
        self.preprocessor_agent = PreprocessorAgent(2, self)
        self.classifier_agent = ClassifierAgent(3, self)
        self.evaluator_agent = EvaluatorAgent(4, self)

        # Add agents to the scheduler
        self.scheduler.add(self.informer_agent)
        self.scheduler.add(self.preprocessor_agent)
        self.scheduler.add(self.classifier_agent)
        self.scheduler.add(self.evaluator_agent)

    def step(self):
        self.scheduler.step()

# Create and run the model
initial_string = "This product is amazing!"
model = TextClassificationModel(initial_string)
model.step()


<ipython-input-5-860532a3cdf6>:9: FutureWarning: The Mesa Model class was not initialized. In the future, you need to explicitly initialize the Model by calling super().__init__() on initialization.
  super().__init__(unique_id, model)


NotFittedError: Vocabulary not fitted or provided